In [35]:
import os
import pandas as pd
import numpy as np
import gensim 
import tqdm   #to visualize loops' progress

In [36]:
## prepare the path variables
root_path = os.getcwd()
data_dir = os.path.join(root_path,'Data','datasets')
new_data_dir = os.path.join(root_path,'Data','datasets','new datasets')
if not os.path.isdir(new_data_dir):
    os.mkdir(new_data_dir)

In [37]:
## Import Data from csv
# read Users
u_cols = ['user_id', 'location', 'age']
users = pd.read_csv(os.path.join(data_dir,'BX_Users.csv'), sep=';', names=u_cols, encoding='latin-1',low_memory=False)

# read Books/items
i_cols = ['isbn', 'book_title' ,'book_author','year_of_publication', 'publisher', 'img_s', 'img_m', 'img_l']
items = pd.read_csv(os.path.join(data_dir,'BX_Books.csv'), sep=';', names=i_cols, encoding='latin-1',low_memory=False)

# read Ratings
r_cols = ['user_id', 'isbn', 'rating']
ratings = pd.read_csv(os.path.join(data_dir,'BX_Book_Ratings.csv'), sep=';', names=r_cols, encoding='latin-1',low_memory=False)

In [38]:
print('__________________________________Users__________________________________')
print(users.head(3))
print("\n")
print('__________________________________Items__________________________________')
print(items.head(3))
print("\n")
print('_________________________________Ratings_________________________________')
print(ratings.head(3))

__________________________________Users__________________________________
   user_id                   location  age
0  User-ID                   Location  Age
1        1         nyc, new york, usa  NaN
2        2  stockton, california, usa   18


__________________________________Items__________________________________
         isbn           book_title           book_author  year_of_publication  \
0        ISBN           Book-Title           Book-Author  Year-Of-Publication   
1  0195153448  Classical Mythology    Mark P. O. Morford                 2002   
2  0002005018         Clara Callan  Richard Bruce Wright                 2001   

                 publisher                                              img_s  \
0                Publisher                                        Image-URL-S   
1  Oxford University Press  http://images.amazon.com/images/P/0195153448.0...   
2    HarperFlamingo Canada  http://images.amazon.com/images/P/0002005018.0...   

                            

<p></p><p style="font-size:15pt">删除不需要的列</p>


In [39]:
users = users.loc[1:]
users.reset_index(drop=True, inplace=True)

items = items.loc[1:]
items.reset_index(drop=True, inplace=True)
items.drop(['img_s','img_l'], axis=1, inplace=True)

ratings = ratings.loc[1:]
ratings.reset_index(drop=True, inplace=True)

print('__________________________________Users__________________________________')
print(users.head(5))
print("\n")
print('__________________________________Items__________________________________')
print(items.head(5))
print("\n")
print('_________________________________Ratings_________________________________')
print(ratings.head(5))

__________________________________Users__________________________________
  user_id                            location  age
0       1                  nyc, new york, usa  NaN
1       2           stockton, california, usa   18
2       3     moscow, yukon territory, russia  NaN
3       4           porto, v.n.gaia, portugal   17
4       5  farnborough, hants, united kingdom  NaN


__________________________________Items__________________________________
         isbn                                         book_title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            book_author year_of_publication                   publisher  \
0    Mark P. O. Morford                2002     Oxford Unive

<p></p><p style="font-size:15pt">查看数据</p>

In [40]:
print(users.describe(),"\n"*2)
print(items.describe(),"\n"*2)
print(ratings.describe())

       user_id                         location     age
count   278858                           278858  168096
unique  278858                            57339     165
top     104278  london, england, united kingdom      24
freq         1                             2506    5687 


              isbn      book_title      book_author year_of_publication  \
count       271379          271379           271378              271379   
unique      271379          242154           102042                 137   
top     0895772256  Selected Poems  Agatha Christie                2002   
freq             1              27              632               17627   

        publisher                                              img_m  
count      271377                                             271379  
unique      16824                                             271063  
top     Harlequin  http://images.amazon.com/images/P/043935806X.0...  
freq         7535                                        

In [41]:
users.age = users.age.astype(float)
users.user_id = users.user_id.astype(int)
users.describe(include=[object, int, float])

,user_id,location,age
count,278858.00000,278858,168096.000000
unique,NaN,57339,NaN
top,NaN,"london, england, united kingdom",NaN
freq,NaN,2506,NaN
mean,139429.50000,NaN,34.751434
std,80499.51502,NaN,14.428097
min,1.00000,NaN,0.000000
25%,69715.25000,NaN,24.000000
50%,139429.50000,NaN,32.000000
75%,209143.75000,NaN,44.000000


<br></br><h2 style="font-size:20pt;"> users.age</h2>

<p style="font-size:15pt"> Let's take a closer look at "users.age" Series. As we can see from above it has a mean value around 34.75 and a deviation of 14.43. Next, we shall investigate how many NaN values there are in the Series and how many values are "not so logical" (> 5 or < 99).</p>

In [42]:
print("NaN values in age:", users.age[users.age.isna()].count())
print("Users with 5 > age & age > 99 :",users.loc[(users.age>99) | (users.age<5),'age'].count())
users.loc[(users.age>99) | (users.age<5),'age'] = np.nan

NaN values in age: 0
Users with 5 > age & age > 99 : 1255


<p style="font-size:15pt"><b>Wow</b>, 110762 missing values in <em>users.age</em> and another 1255 users have <i>age</i> values that I think are invalid. <br></br>Since we really don't like NaN values in datasets we will replace them with a value. This value can be the mean age of the users that registered their age or it can be based on some values of the dataset. For example, we can use the location of the user and use the average age of the population of this location, or we can use the average age of the population using PCs on that location since this dataset was collected from a website. We can even use an ML algorithm to determine the missing values. The easiest way  would be to use the mean.</p>

In [43]:
users.age.fillna(users.age.mean()).describe()

count    278858.000000
mean         34.743900
std          10.540292
min           5.000000
25%          29.000000
50%          34.743900
75%          35.000000
max          99.000000
Name: age, dtype: float64

In [44]:
# create a normal disgtribution pd.Series to fill Nan values with
temp_age_series = pd.Series(np.random.normal(loc=users.age.mean(), scale=users.age.std(), size=users.user_id[users.age.isna()].count()))
print("Statistics of values in \'users.age\'\n",users.age.describe(),"\n")
print("Statistics of values we are going to use to fill NaN \n",temp_age_series.describe(),"\n")
print("Negative values in \'temp_age_seires\':", temp_age_series[temp_age_series<0].count(),"\n")
print("As we can see the destribution doesnt change a lot. There are some negative values thought (around 600 of them).\n")

# take the abs value of temp_age_series
pos_age_series=np.abs(temp_age_series)

# sort users Df so as NaN values in age to be first and reset index to match with index of pos_age_series. Then use fillna()
users = users.sort_values('age',na_position='first').reset_index(drop=True)
users.age.fillna(pos_age_series, inplace = True)  

# replace values < 5 with the mean(). Round values and convert them to int. 
users.loc[users.age<5, 'age'] = users.age.mean()
users.age = users.age.round().astype(int)
#Sort users based on user_id so as to be the same as before
users = users.sort_values('user_id').reset_index(drop=True)
print(users.age.describe(),"\n")
users.head()

Statistics of values in 'users.age'
 count    166841.000000
mean         34.743900
std          13.626783
min           5.000000
25%          24.000000
50%          32.000000
75%          44.000000
max          99.000000
Name: age, dtype: float64 

Statistics of values we are going to use to fill NaN 
 count    112017.000000
mean         34.727375
std          13.645074
min         -26.551841
25%          25.551266
50%          34.738286
75%          43.941889
max          96.612554
dtype: float64 

Negative values in 'temp_age_seires': 608 

As we can see the destribution doesnt change a lot. There are some negative values thought (around 600 of them).

count    278858.000000
mean         34.921591
std          13.390613
min           5.000000
25%          25.000000
50%          33.000000
75%          44.000000
max          99.000000
Name: age, dtype: float64 



,user_id,location,age
0,1,"nyc, new york, usa",24
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",24
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",63


In [45]:
users.location.head()

0                    nyc, new york, usa
1             stockton, california, usa
2       moscow, yukon territory, russia
3             porto, v.n.gaia, portugal
4    farnborough, hants, united kingdom
Name: location, dtype: object

In [46]:
location_split=users.location.str.split(', ', n=2, expand=True)
location_split.columns=['city', 'state', 'country']
location_split.describe(include=[object])

,city,state,country
count,278858,278857,274281
unique,33076,6663,1130
top,london,california,usa
freq,4105,19839,139421


In [47]:
location_split.loc[location_split.state==',', ['state', 'country']] = 'other'
location_split.loc[location_split.country==',', ['country']] = 'other'
location_split.loc[(location_split.state=='\\n/a\\"') | (location_split.state=='') | (location_split.state=='*') | (location_split.state=='n.a'), ['state']] = 'n/a'
location_split.state.fillna('other', inplace=True)
location_split.fillna('n/a', inplace=True)

In [48]:
temp_location_df = pd.concat([location_split.city, location_split.state,  location_split.country, location_split.state, location_split.city, location_split.country, location_split.city], axis=1)
location_list = temp_location_df.fillna('n/a').values.tolist()

In [49]:
n = 10
model = gensim.models.Word2Vec(location_list, size= n, window=3, min_count=1, workers=4)
print ('UK is to Milton Keynes what Greece is to : ')
model.wv.most_similar(positive=['greece','united kingdom'], negative=['milton keynes'], topn=20)

UK is to Milton Keynes what Greece is to : 


[('crete', 0.9693020582199097),
 ('piraeus', 0.9634312987327576),
 ('attiki', 0.9616715908050537),
 ('papagou', 0.9522576332092285),
 ('attika', 0.9521435499191284),
 ('maroussi', 0.95106041431427),
 ('thessaloniki', 0.9464523792266846),
 ('basilika', 0.943909764289856),
 ('attica', 0.940754234790802),
 ('patras', 0.940321683883667),
 ('arequipa', 0.9388856291770935),
 ('denizli', 0.9370570182800293),
 ('chery hill', 0.9368599653244019),
 ('drama', 0.9359506368637085),
 ('serres', 0.934550404548645),
 ('magnisia', 0.933468759059906),
 ('trujillo', 0.9325399398803711),
 ('lavrio', 0.932529091835022),
 ('aten', 0.930486798286438),
 ('athens', 0.9298406839370728)]

In [50]:
rightchoice=['1','2']
choice = input("Choose \'1\' to skip this step or \'2\' to construct the \'location_vec\' DataFrame.")
while choice not in rightchoice:
    choice = input("Wrong input. \n Insert a number. Either 1 or 2")
if choice=='1':
    print ('Skipping operations')
else:
    zipp = list(zip(model.wv.index2word, model.wv.syn0))
    vectors = np.zeros((location_split.shape[0],3*n))
    for i in tqdm.tqdm_notebook(range(location_split.shape[0])):
        vectors[i, 0:20] = [j[1][0] for j in zipp if j[0] == location_split.loc[i, 'city']]
        vectors[i,20:40] = [j[1][0] for j in zipp if j[0] == location_split.loc[i, 'state']]
        vectors[i,40:60] = [j[1][0] for j in zipp if j[0] == location_split.loc[i, 'country']]
    col=[]
    for i in range(20):
        col.append('city_'+ str(i))
    for i in range(20):
        col.append('state_'+ str(i))
    for i in range(20):
        col.append('country_'+ str(i))

    location_vec = pd.DataFrame(vectors, columns = col)

Choose '1' to skip this step or '2' to construct the 'location_vec' DataFrame.1
Skipping operations


In [51]:
if 'location_vec' in globals():
    users_new = pd.concat([users.user_id, location_vec , users.age], axis=1)    
else:
    users_new = pd.concat([users.user_id, location_split , users.age], axis=1)
users_new.head()

,user_id,city,state,country,age
0,1,nyc,new york,usa,24
1,2,stockton,california,usa,18
2,3,moscow,yukon territory,russia,24
3,4,porto,v.n.gaia,portugal,17
4,5,farnborough,hants,united kingdom,63


In [52]:
print(users_new[users_new.user_id.isin(ratings.user_id)].user_id.count(),'users have submited at least one review')
print(users_new[~users_new.user_id.isin(ratings.user_id)].user_id.count(), 'users have not submited any review')

105283 users have submited at least one review
173575 users have not submited any review


In [53]:
items = pd.read_csv(os.path.join(data_dir,'BX_Books_correct.csv'), sep=';', names=i_cols, encoding='latin-1',low_memory=False)
items = items.loc[1:]
items.reset_index(drop=True, inplace=True)
items.drop(['img_s','img_l'], axis=1, inplace=True)
items.year_of_publication = items.year_of_publication.astype(int)
items.describe(include =[object, int])
items.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_m
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...


In [54]:
print ("Items with NaN values in \"book_author\": \n", items.isbn[items.book_author.isna()],"\n")
print ("Items values in \"publisher\": \n", items.isbn[items.publisher.isna()])

Items with NaN values in "book_author": 
 187700    9627982032
Name: isbn, dtype: object 

Items values in "publisher": 
 128896    193169656X
129043    1931696993
Name: isbn, dtype: object


In [55]:
items.loc[187701,'book_author'] = "n/a"
items.loc[[128897, 129044],'publisher'] = "NovelBooks, Inc"

In [56]:
print('Items with (year_of_publication > 2010):', items.year_of_publication[items.year_of_publication>2010].count(),'\n')
print('value_counts of items with (year_of_publication < 1500): \n', items.year_of_publication[items.year_of_publication<1500].value_counts())

Items with (year_of_publication > 2010): 20 

value_counts of items with (year_of_publication < 1500): 
 0       4619
1378       1
1376       1
Name: year_of_publication, dtype: int64


In [57]:
items.loc[(items.year_of_publication>2010)|(items.year_of_publication<1000),'year_of_publication'] = np.nan
print(items.describe(),'\n')
print(items.year_of_publication.fillna(round(items.year_of_publication.mean())).describe())

       year_of_publication
count        266740.000000
mean           1993.687062
std               8.320673
min            1376.000000
25%            1989.000000
50%            1996.000000
75%            2000.000000
max            2010.000000 

count    271379.000000
mean       1993.692412
std           8.249348
min        1376.000000
25%        1989.000000
50%        1995.000000
75%        2000.000000
max        2010.000000
Name: year_of_publication, dtype: float64


In [58]:
items.year_of_publication.fillna(round(items.year_of_publication.mean()),inplace=True)
items.year_of_publication = items.year_of_publication.astype(int)

In [59]:
print(items[(items.duplicated(['book_title', 'book_author'], keep=False))].describe(include=[object]))
print(items[(items.duplicated(['book_title', 'book_author'], keep='first'))].describe(include=[object]))
print(items[(items.duplicated(['book_title', 'book_author']))].book_author.value_counts().head())

              isbn    book_title      book_author         publisher  \
count        35921         35921            35921             35921   
unique       35921         15376             7694              2505   
top     0446612588  Little Women  Agatha Christie  Ballantine Books   
freq             1            21              259              1095   

                                                    img_m  
count                                               35921  
unique                                              35606  
top     http://images.amazon.com/images/P/067103619X.0...  
freq                                                    2  
              isbn    book_title   book_author         publisher  \
count        20175         20175         20175             20175   
unique       20175         15376          7694              2019   
top     0441385516  Little Women  Stephen King  Ballantine Books   
freq             1            20           192               590   

   

In [60]:
items_wo_duplicates = items.drop_duplicates(['book_title', 'book_author'])
items_wo_duplicates.describe(include=[object,int])

,isbn,book_title,book_author,year_of_publication,publisher,img_m
count,251204,251204,251203,251204.000000,251202,251204
unique,251204,242154,102029,NaN,16617,251203
top,0895772256,Selected Poems,William Shakespeare,NaN,Harlequin,http://images.amazon.com/images/P/051513628X.0...
freq,1,25,496,NaN,7508,2
mean,NaN,NaN,NaN,1993.705817,NaN,NaN
std,NaN,NaN,NaN,8.245138,NaN,NaN
min,NaN,NaN,NaN,1376.000000,NaN,NaN
25%,NaN,NaN,NaN,1989.000000,NaN,NaN
50%,NaN,NaN,NaN,1995.000000,NaN,NaN
75%,NaN,NaN,NaN,2000.000000,NaN,NaN


In [61]:
ratings_new = ratings[ratings.isbn.isin(items.isbn)]
ratings_new.describe()

,user_id,isbn,rating
count,1031175,1031175,1031175
unique,92107,270170,11
top,11676,0971880107,0
freq,11144,2502,647323


In [62]:
ratings_new.loc[:,'rating'] = ratings_new.rating.astype(int)
print(ratings_new.rating.value_counts(sort=False))
ratings_new.describe(include=[object,int])

E:\anaconda\lib\site-packages\pandas\core\indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


0     647323
1       1481
2       2375
3       5118
4       7617
5      45355
6      31689
7      66404
8      91806
9      60780
10     71227
Name: rating, dtype: int64


,user_id,isbn,rating
count,1031175,1031175,1.031175e+06
unique,92107,270170,NaN
top,11676,0971880107,NaN
freq,11144,2502,NaN
mean,NaN,NaN,2.839022e+00
std,NaN,NaN,3.854149e+00
min,NaN,NaN,0.000000e+00
25%,NaN,NaN,0.000000e+00
50%,NaN,NaN,0.000000e+00
75%,NaN,NaN,7.000000e+00


In [63]:
choice = input("Choose \'1\' to import the ratings_wo_duplicates file or \'2\' to construct it again ")
while choice not in rightchoice:
    choice = input("Wrong input. \n Insert a number. Either 1 or 2")

if choice == '1':
    print('Importing \'ratings_wo_duplicates.csv\'')
    ratings_wo_duplicates=pd.read_csv(os.path.join(data_dir,'ratings_wo_duplicates.csv'), sep=';', names=r_cols, encoding='latin-1', low_memory=False )
    print('Done')
elif choice == '2':
    print('Constructing \'ratings_wo_duplicates.csv\'')
    print('Please remember the number of processed and stored items incase the operation is interupted and you would like continue from there.')
    
    choice = input("Choose \'1\' to iterate through all items or \'2\' if this operation was interupted and you would like to continue from the last checkpoint.")
    while choice not in rightchoice:
        choice = input("Wrong input. \n Insert a number. Either 1 or 2")
    
    if choice == '1':
        nof = 0
        ratings_wo_duplicates = ratings_new
        count=0
    else:
        nof = int(input('Please insert the number of processed and stored items.'))
        ratings_wo_duplicates=pd.read_csv(os.path.join(new_data_dir,'ratings_wo_duplicates.csv'), sep=';', names=r_cols, encoding='latin-1', low_memory=False )
        count= nof
    
    # create a series with all the duplicates (including the first occurance) to iterate
    temp=items[(items.duplicated(['book_title', 'book_author'],keep=False))][nof:]
    
    for t in tqdm.tqdm_notebook(temp['book_title']):
        x = list( items[items['book_title']==t].isbn)
        count+=1 
        for i in range(1, len(x)):
            #replace all entries in x list with x[0] (the isbn we kept in items_wo_duplicates)
            ratings_wo_duplicates.loc[ratings_wo_duplicates.isbn==x[i],'isbn'] = x[0]

        if count%2000==0:
            ratings_wo_duplicates.to_csv(os.path.join(new_data_dir,'ratings_wo_duplicates.csv'),';', index=False)
            print(count ,' duplicate items ratings processed and stored')
            
    ratings_wo_duplicates.to_csv(os.path.join(new_data_dir,'ratings_wo_duplicates.csv'),';', index=False)
    print('Done')


ratings_wo_duplicates = ratings_wo_duplicates.loc[1:]
ratings_wo_duplicates.reset_index(drop=True, inplace=True)
ratings_wo_duplicates.rating = ratings_wo_duplicates.rating.astype(int)
print('\nAnd to make sure that the procedure was carried out smoothly,')
print('No of duplicates in \"ratings_wo_duplicates\" :',ratings_wo_duplicates.isbn[ratings_wo_duplicates.isbn.isin(items[items.duplicated(['book_title', 'book_author'])].isbn)].count()) 

Choose '1' to import the ratings_wo_duplicates file or '2' to construct it again 1
Importing 'ratings_wo_duplicates.csv'
Done

And to make sure that the procedure was carried out smoothly,
No of duplicates in "ratings_wo_duplicates" : 0


In [64]:
ratings_expl = ratings_wo_duplicates[ratings_wo_duplicates.rating != 0]
ratings_impl = ratings_wo_duplicates[ratings_wo_duplicates.rating == 0]

In [65]:
print(ratings_expl.describe(include=[object,int]),'\n')
print(ratings_impl.describe(include=[object,int]))

       user_id        isbn         rating
count   383852      383852  383852.000000
unique   68092      137646            NaN
top      11676  0316666343            NaN
freq      6943         707            NaN
mean       NaN         NaN       7.626710
std        NaN         NaN       1.841331
min        NaN         NaN       1.000000
25%        NaN         NaN       7.000000
50%        NaN         NaN       8.000000
75%        NaN         NaN       9.000000
max        NaN         NaN      10.000000 

       user_id        isbn    rating
count   647322      647322  647322.0
unique   52451      184258       NaN
top     198711  0971880107       NaN
freq      6439        1921       NaN
mean       NaN         NaN       0.0
std        NaN         NaN       0.0
min        NaN         NaN       0.0
25%        NaN         NaN       0.0
50%        NaN         NaN       0.0
75%        NaN         NaN       0.0
max        NaN         NaN       0.0


In [66]:
users_w_ex_ratings = users_new[users_new.user_id.isin(ratings_expl.user_id)]
users_w_im_ratings = users_new[users_new.user_id.isin(ratings_impl.user_id)]

In [67]:
items_w_ratings = items_wo_duplicates[items_wo_duplicates.isbn.isin(ratings_wo_duplicates.isbn)]

In [68]:
users_new.to_csv(os.path.join(new_data_dir,'users_new.csv'),';', index=False)
users_w_ex_ratings.to_csv(os.path.join(new_data_dir,'users_w_ex_ratings.csv'),';', index=False)
users_w_im_ratings.to_csv(os.path.join(new_data_dir,'users_w_im_ratings.csv'),';', index=False)
items_wo_duplicates.to_csv(os.path.join(new_data_dir,'items_wo_duplicates.csv'),';', index=False)
ratings_wo_duplicates.to_csv(os.path.join(new_data_dir,'ratings_wo_duplicates.csv'),';', index=False)
ratings_expl.to_csv(os.path.join(new_data_dir,'ratings_expl.csv'),';', index=False)
ratings_impl.to_csv(os.path.join(new_data_dir,'ratings_impl.csv'),';', index=False)
print("DONE!!")

DONE!!
